In [1]:
import os
import re
import sys
import time
import implicit
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy import sparse
from tqdm import tqdm_notebook as tqdm

In [2]:
IS_LOCAL = True
HOME_DIR = '/mnt/E/Projects/Content-based-Neural-Recommender-Systems/' if IS_LOCAL else '../../'
os.environ['HOME_DIR'] = HOME_DIR

sys.path.append("../..")
from utils.prepare_data import get_zen_data, zen_text_iterator

DATA_DIR = f'{HOME_DIR}data/zen/'
WORKING_DIR = f'{HOME_DIR}/models/als/'
os.chdir(WORKING_DIR)

In [3]:
!pwd

/mnt/E/Projects/Content-based-Neural-Recommender-Systems/models/als


## Load data

In [4]:
items_df, (train_df, test_df) = get_zen_data()

loading items: 328050it [00:31, 10496.56it/s]
loading users: 42977it [02:44, 260.79it/s]


In [6]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
doc2vec_model = Doc2Vec(documents=zen_text_iterator(), vector_size=64, window=10, workers=4, epochs=5)

2019-04-06 19:09:40,441 : INFO : collecting all words and their counts
2019-04-06 19:09:40,443 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-04-06 19:09:43,251 : INFO : PROGRESS: at example #10000, processed 2098645 words (747450/s), 194725 word types, 10000 tags
2019-04-06 19:09:46,079 : INFO : PROGRESS: at example #20000, processed 4206669 words (745660/s), 279127 word types, 20000 tags
2019-04-06 19:09:48,897 : INFO : PROGRESS: at example #30000, processed 6310699 words (746880/s), 340949 word types, 30000 tags
2019-04-06 19:09:51,790 : INFO : PROGRESS: at example #40000, processed 8434291 words (734192/s), 393353 word types, 40000 tags
2019-04-06 19:09:54,614 : INFO : PROGRESS: at example #50000, processed 10542423 words (746954/s), 437816 word types, 50000 tags
2019-04-06 19:09:57,445 : INFO : PROGRESS: at example #60000, processed 12652322 words (745469/s), 477648 word types, 60000 tags
2019-04-06 19:10:00,288 : INFO : PROGRESS: at example #

2019-04-06 19:11:50,401 : INFO : EPOCH 1 - PROGRESS: at 17.74% examples, 416414 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:51,405 : INFO : EPOCH 1 - PROGRESS: at 18.41% examples, 416894 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:52,413 : INFO : EPOCH 1 - PROGRESS: at 19.05% examples, 416643 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:53,414 : INFO : EPOCH 1 - PROGRESS: at 19.71% examples, 416813 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:54,426 : INFO : EPOCH 1 - PROGRESS: at 20.37% examples, 416838 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:55,449 : INFO : EPOCH 1 - PROGRESS: at 21.04% examples, 417022 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:56,451 : INFO : EPOCH 1 - PROGRESS: at 21.68% examples, 416876 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:11:57,456 : INFO : EPOCH 1 - PROGRESS: at 22.32% examples, 416727 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:11:58,480 : INFO : EPOCH 1 - PROGRESS: at 22.97% examples, 416386 words/s, in_qsiz

2019-04-06 19:13:04,550 : INFO : EPOCH 1 - PROGRESS: at 65.69% examples, 416014 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:13:05,552 : INFO : EPOCH 1 - PROGRESS: at 66.33% examples, 415982 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:06,566 : INFO : EPOCH 1 - PROGRESS: at 66.99% examples, 415991 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:07,578 : INFO : EPOCH 1 - PROGRESS: at 67.63% examples, 415924 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:08,601 : INFO : EPOCH 1 - PROGRESS: at 68.29% examples, 415974 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:09,617 : INFO : EPOCH 1 - PROGRESS: at 68.94% examples, 415968 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:10,632 : INFO : EPOCH 1 - PROGRESS: at 69.60% examples, 415974 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:11,655 : INFO : EPOCH 1 - PROGRESS: at 70.25% examples, 415938 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:13:12,680 : INFO : EPOCH 1 - PROGRESS: at 70.92% examples, 415981 words/s, in_qsiz

2019-04-06 19:14:15,538 : INFO : EPOCH 2 - PROGRESS: at 11.98% examples, 420709 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:16,543 : INFO : EPOCH 2 - PROGRESS: at 12.62% examples, 420664 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:17,544 : INFO : EPOCH 2 - PROGRESS: at 13.28% examples, 420743 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:18,565 : INFO : EPOCH 2 - PROGRESS: at 13.95% examples, 420814 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:19,572 : INFO : EPOCH 2 - PROGRESS: at 14.60% examples, 420800 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:20,587 : INFO : EPOCH 2 - PROGRESS: at 15.27% examples, 420645 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:14:21,589 : INFO : EPOCH 2 - PROGRESS: at 15.94% examples, 421088 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:22,596 : INFO : EPOCH 2 - PROGRESS: at 16.59% examples, 421007 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:14:23,627 : INFO : EPOCH 2 - PROGRESS: at 17.27% examples, 420948 words/s, in_qsiz

2019-04-06 19:15:29,719 : INFO : EPOCH 2 - PROGRESS: at 60.52% examples, 420782 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:15:30,750 : INFO : EPOCH 2 - PROGRESS: at 61.20% examples, 420849 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:15:31,766 : INFO : EPOCH 2 - PROGRESS: at 61.87% examples, 420892 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:15:32,776 : INFO : EPOCH 2 - PROGRESS: at 62.54% examples, 420956 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:15:33,779 : INFO : EPOCH 2 - PROGRESS: at 63.21% examples, 421145 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:15:34,782 : INFO : EPOCH 2 - PROGRESS: at 63.88% examples, 421247 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:15:35,800 : INFO : EPOCH 2 - PROGRESS: at 64.55% examples, 421278 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:15:36,806 : INFO : EPOCH 2 - PROGRESS: at 65.25% examples, 421535 words/s, in_qsize 6, out_qsize 0
2019-04-06 19:15:37,809 : INFO : EPOCH 2 - PROGRESS: at 65.93% examples, 421623 words/s, in_qsiz

...

2019-04-06 19:17:53,965 : INFO : EPOCH 3 - PROGRESS: at 57.34% examples, 431699 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:17:55,004 : INFO : EPOCH 3 - PROGRESS: at 58.03% examples, 431620 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:17:56,034 : INFO : EPOCH 3 - PROGRESS: at 58.72% examples, 431580 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:17:57,063 : INFO : EPOCH 3 - PROGRESS: at 59.39% examples, 431540 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:17:58,070 : INFO : EPOCH 3 - PROGRESS: at 60.07% examples, 431602 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:17:59,071 : INFO : EPOCH 3 - PROGRESS: at 60.73% examples, 431495 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:18:00,072 : INFO : EPOCH 3 - PROGRESS: at 61.41% examples, 431595 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:18:01,072 : INFO : EPOCH 3 - PROGRESS: at 62.08% examples, 431586 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:18:02,080 : INFO : EPOCH 3 - PROGRESS: at 62.76% examples, 431644 words/s, in_qsiz

2019-04-06 19:19:04,294 : INFO : EPOCH 4 - PROGRESS: at 4.83% examples, 436838 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:05,315 : INFO : EPOCH 4 - PROGRESS: at 5.53% examples, 437371 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:06,325 : INFO : EPOCH 4 - PROGRESS: at 6.22% examples, 437348 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:07,331 : INFO : EPOCH 4 - PROGRESS: at 6.90% examples, 437491 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:08,346 : INFO : EPOCH 4 - PROGRESS: at 7.60% examples, 438058 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:09,366 : INFO : EPOCH 4 - PROGRESS: at 8.30% examples, 438311 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:10,396 : INFO : EPOCH 4 - PROGRESS: at 9.01% examples, 438143 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:11,403 : INFO : EPOCH 4 - PROGRESS: at 9.71% examples, 438770 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:19:12,414 : INFO : EPOCH 4 - PROGRESS: at 10.40% examples, 438672 words/s, in_qsize 7, out

2019-04-06 19:20:18,357 : INFO : EPOCH 4 - PROGRESS: at 55.74% examples, 441608 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:20:19,358 : INFO : EPOCH 4 - PROGRESS: at 56.43% examples, 441609 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:20,368 : INFO : EPOCH 4 - PROGRESS: at 57.11% examples, 441545 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:21,370 : INFO : EPOCH 4 - PROGRESS: at 57.81% examples, 441539 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:22,373 : INFO : EPOCH 4 - PROGRESS: at 58.51% examples, 441623 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:23,380 : INFO : EPOCH 4 - PROGRESS: at 59.20% examples, 441679 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:24,383 : INFO : EPOCH 4 - PROGRESS: at 59.88% examples, 441646 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:25,388 : INFO : EPOCH 4 - PROGRESS: at 60.58% examples, 441719 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:20:26,420 : INFO : EPOCH 4 - PROGRESS: at 61.26% examples, 441546 words/s, in_qsiz

2019-04-06 19:21:28,563 : INFO : EPOCH 5 - PROGRESS: at 4.17% examples, 439161 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:21:29,567 : INFO : EPOCH 5 - PROGRESS: at 4.87% examples, 440384 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:21:30,589 : INFO : EPOCH 5 - PROGRESS: at 5.57% examples, 440452 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:21:31,606 : INFO : EPOCH 5 - PROGRESS: at 6.27% examples, 440725 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:21:32,610 : INFO : EPOCH 5 - PROGRESS: at 6.98% examples, 441549 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:21:33,636 : INFO : EPOCH 5 - PROGRESS: at 7.68% examples, 442125 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:21:34,638 : INFO : EPOCH 5 - PROGRESS: at 8.38% examples, 442651 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:21:35,655 : INFO : EPOCH 5 - PROGRESS: at 9.09% examples, 442607 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:21:36,662 : INFO : EPOCH 5 - PROGRESS: at 9.79% examples, 442925 words/s, in_qsize 7, out_

2019-04-06 19:22:42,637 : INFO : EPOCH 5 - PROGRESS: at 55.37% examples, 444077 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:22:43,686 : INFO : EPOCH 5 - PROGRESS: at 56.08% examples, 443996 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:22:44,721 : INFO : EPOCH 5 - PROGRESS: at 56.82% examples, 444105 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:22:45,741 : INFO : EPOCH 5 - PROGRESS: at 57.51% examples, 444072 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:22:46,775 : INFO : EPOCH 5 - PROGRESS: at 58.23% examples, 444083 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:22:47,780 : INFO : EPOCH 5 - PROGRESS: at 58.94% examples, 444233 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:22:48,793 : INFO : EPOCH 5 - PROGRESS: at 59.63% examples, 444232 words/s, in_qsize 8, out_qsize 0
2019-04-06 19:22:49,795 : INFO : EPOCH 5 - PROGRESS: at 60.33% examples, 444292 words/s, in_qsize 7, out_qsize 0
2019-04-06 19:22:50,804 : INFO : EPOCH 5 - PROGRESS: at 61.03% examples, 444303 words/s, in_qsiz

In [54]:
def process_item(item_id):
    item = items_df.iloc[item_id]
    return np.concatenate((doc2vec_model.infer_vector(tokenize('{} {}'.format(item["title"], item["content"]))), item['image']))

In [75]:
def get_random_responses_generator(users_df):
    while True:
        user_id, items, ratings = users_df.iloc[np.random.randint(0, users_df.shape[0])]
        rnd_i = np.random.randint(0, len(items))
        item_id = items[rnd_i]
        rating = ratings[rnd_i]
        yield user_id, process_item(item_id), rating

In [76]:
def get_responses_generator(users_df):
    for _, (user_id, items, ratings) in users_df.iterrows():
        for item_id, rating in zip(items, ratings):
            yield user_id, process_item(item_id), rating

In [281]:
def batch_generator(responses_generator, batch_size=32):
    user_ids, user_items, user_ratings = [], [], []
    i = -1
    for user_id, user_item, user_rating in responses_generator:
        user_ids.append(user_id)
        user_items.append(user_item)
        user_ratings.append(user_rating)
        if i % batch_size == 0:
            yield tuple(map(np.array, [user_ids, user_items, user_ratings]))
            user_ids, user_items, user_ratings = [], [], []
        i += 1

In [311]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    # close current session
    if curr_session is not None:
        curr_session.close()
    # reset graph
    tf.reset_default_graph()
    # create new session
#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     config.allow_soft_placement=True
#     config.log_device_placement=True
#     s = tf.InteractiveSession(config=config)
#     return s

In [514]:
class MLP():
    def __init__(self, doc2vec_model, n_users, user_embedding_size, item_embedding_size, hidden_size, n_layers, sess):
        self.doc2vec_model = doc2vec_model
        self.n_users = n_users
        self.user_embedding_size = user_embedding_size
        self.item_embedding_size = item_embedding_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.sess = sess
#         self.n_items = n_items

    def define_graph(self, learning_rate):
        self.user_id = tf.placeholder(tf.int32, shape=[None], name='user_id')
        self.user_embeddings = tf.get_variable(
            'user_embeddings',
            shape=[self.n_users, self.user_embedding_size],
            dtype=tf.float32
        )
        print(self.user_id)
        print(self.user_embeddings)
        user_embedding = tf.nn.embedding_lookup(
            self.user_embeddings, self.user_id)
        print(user_embedding)
        self.item_embedding = tf.placeholder(
            tf.float32, shape=[None, self.item_embedding_size])
        print(self.item_embedding)
        self.y = tf.placeholder(tf.float32, name='y')
        

        layer = tf.concat([user_embedding, self.item_embedding], axis=1)
        print(layer)
        for i in np.arange(self.n_layers):
            layer = tf.layers.dense(
                layer, self.hidden_size, tf.nn.elu, kernel_initializer=tf.random_normal_initializer)
        self.p = tf.layers.dense(layer, 1, tf.nn.sigmoid, kernel_initializer=tf.random_normal_initializer)
        print(self.p)
        self.loss = -tf.reduce_sum(self.y * tf.log(self.p + 1e-10) + (1 - self.y) * tf.log(1 - self.p + 1e-10))
        print(self.loss)
        self.optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(self.loss)
    
    def train(self, train_df, steps):
        train_ratings = get_random_responses_generator(train_df)
        loss = []
        for it, (batch_user, batch_item, batch_rating) in enumerate(batch_generator(train_ratings)):
            if (it + 1) % steps == 0:
                break
#             batch_rating[batch_rating == 0] = -1
            _, l = self.sess.run((self.optimizer, self.loss),
                                feed_dict={self.user_id: batch_user, self.item_embedding: batch_item, self.y: batch_rating})
#                 print("Iteration: %04d; cost= %.9f" % (it, np.mean(loss)))
            loss.append(l)
            if it % 100 == 0:
                print(f'Iteration: {it}, loss: {np.mean(loss)}')
#                 print(self.predict(batch_user, batch_item))
#                 print(batch_rating)
                

    def execute(self, train_df, epochs, epoch_steps, learning_rate):
#         self.sess = reset_tf_session()
        self.define_graph(learning_rate)
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
        for epoch in np.arange(epochs)+1:
            self.train(train_df, epoch_steps)
            print("Epoch: %04d; " % (epoch), end='')
#             self.test(test_df)
            
    
    def test(self, test_df):
        loss = []
        for user_id, item, rating in tqdm(batch_generator(get_responses_generator(test_df))):
            p = self.predict(user_id, item)
            loss.append(rating * np.log(p) + (1 - rating) * tf.log(1 - p))
        return np.mean(loss)
    
    
    def predict(self, user_id, item):
        return self.sess.run([self.p], feed_dict={self.user_id: user_id, self.item_embedding: item})[0]

In [515]:
epoch_steps = sum(map(len, train_df['userItems'].values))
epoch_steps

54189923

In [526]:
96+32

128

In [ ]:
reset_tf_session()
sess = tf.Session(config=config)
model = MLP(doc2vec_model, train_df.shape[0], 64, 128, 128, 1, sess)
model.execute(train_df, 10, epoch_steps, 0.01)

In [151]:
import tensorflow as tf
import time
import numpy as np
import random

from utils.evaluation.RankingMetrics import *

class MLP():
    def __init__(self, sess, num_user, num_item, learning_rate=0.5, reg_rate=0.001, epoch=500, batch_size=256,
                 verbose=False, T=5, display_step=1000):
        self.learning_rate = learning_rate
        self.epochs = epoch
        self.batch_size = batch_size
        self.reg_rate = reg_rate
        self.sess = sess
        self.num_user = num_user
        self.num_item = num_item
        self.verbose = verbose
        self.T = T
        self.display_step = display_step
        print("You are running MLP.")

    def build_network(self, num_factor_mlp=10, hidden_dimension=10, num_neg_sample=2):
        self.num_neg_sample = num_neg_sample
        self.user_id = tf.placeholder(dtype=tf.int32, shape=[None], name='user_id')
        self.item_id = tf.placeholder(dtype=tf.int32, shape=[None], name='item_id')
        self.y = tf.placeholder(dtype=tf.float32, shape=[None], name='y')

        self.mlp_P = tf.Variable(tf.random_normal([self.num_user, num_factor_mlp]), dtype=tf.float32)
        self.mlp_Q = tf.Variable(tf.random_normal([self.num_item, num_factor_mlp]), dtype=tf.float32)

        mlp_user_latent_factor = tf.nn.embedding_lookup(self.mlp_P, self.user_id)
        mlp_item_latent_factor = tf.nn.embedding_lookup(self.mlp_Q, self.item_id)

        layer_1 = tf.layers.dense(inputs=tf.concat([mlp_item_latent_factor, mlp_user_latent_factor], axis=1),
                                  units=num_factor_mlp * 2, kernel_initializer=tf.random_normal_initializer,
                                  activation=tf.nn.relu,
                                  kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=self.reg_rate))
        layer_2 = tf.layers.dense(inputs=layer_1, units=hidden_dimension * 2, activation=tf.nn.relu,
                                  kernel_initializer=tf.random_normal_initializer,
                                  kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=self.reg_rate))
        MLP = tf.layers.dense(inputs=layer_2, units=hidden_dimension, activation=tf.nn.relu,
                              kernel_initializer=tf.random_normal_initializer,
                              kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=self.reg_rate))

        self.pred_y = tf.nn.sigmoid(tf.reduce_sum(MLP, axis=1))

        # self.pred_y = tf.layers.dense(inputs=MLP, units=1, activation=tf.sigmoid)

        self.loss = - tf.reduce_sum(
            self.y * tf.log(self.pred_y + 1e-10) + (1 - self.y) * tf.log(1 - self.pred_y + 1e-10)) \
                    + tf.losses.get_regularization_loss() + self.reg_rate * (
        tf.nn.l2_loss(self.mlp_P) + tf.nn.l2_loss(self.mlp_Q))

        self.optimizer = tf.train.AdagradOptimizer(self.learning_rate).minimize(self.loss)

        return self

    def prepare_data(self, train_data, test_data):
        '''
        You must prepare the data before train and test the model
        :param train_data:
        :param test_data:
        :return:
        '''
        t = train_data.tocoo()
        self.user = list(t.row.reshape(-1))
        self.item = list(t.col.reshape(-1))
        self.label = list(t.data)
        self.test_data = test_data

        self.neg_items = self._get_neg_items(train_data.tocsr())
        self.test_users = set([u for u in self.test_data.keys() if len(self.test_data[u]) > 0])

        print("data preparation finished.")
        return self

    def train(self):

        item_temp = self.item[:]
        user_temp = self.user[:]
        labels_temp = self.label[:]

        user_append = []
        item_append = []
        values_append = []
        for u in self.user:
            list_of_random_items = random.sample(self.neg_items[u], self.num_neg_sample)
            user_append += [u] * self.num_neg_sample
            item_append += list_of_random_items
            values_append += [0] * self.num_neg_sample

        item_temp += item_append
        user_temp += user_append
        labels_temp += values_append

        self.num_training = len(item_temp)
        self.total_batch = int(self.num_training / self.batch_size)
        # print(self.total_batch)
        idxs = np.random.permutation(self.num_training)  # shuffled ordering
        user_random = list(np.array(user_temp)[idxs])
        item_random = list(np.array(item_temp)[idxs])
        labels_random = list(np.array(labels_temp)[idxs])

        # train
        for i in range(self.total_batch):
            start_time = time.time()
            batch_user = user_random[i * self.batch_size:(i + 1) * self.batch_size]
            batch_item = item_random[i * self.batch_size:(i + 1) * self.batch_size]
            batch_label = labels_random[i * self.batch_size:(i + 1) * self.batch_size]

            _, loss = self.sess.run((self.optimizer, self.loss),
                                    feed_dict={self.user_id: batch_user, self.item_id: batch_item, self.y: batch_label})

            if i % self.display_step == 0:
                if self.verbose:
                    print("Index: %04d; cost= %.9f" % (i + 1, np.mean(loss)))
                    print("one iteration: %s seconds." % (time.time() - start_time))

    def test(self):
        evaluate(self)

    def execute(self, train_data, test_data):

        self.prepare_data(train_data, test_data)

        init = tf.global_variables_initializer()
        self.sess.run(init)

        for epoch in range(self.epochs):
            self.train()
            if (epoch) % self.T == 0:
                print("Epoch: %04d; " % (epoch), end='')
                self.test()

    def save(self, path):
        saver = tf.train.Saver()
        saver.save(self.sess, path)

    def predict(self, user_id, item_id):
        return self.sess.run([self.pred_y], feed_dict={self.user_id: user_id, self.item_id: item_id})[0]

    def _get_neg_items(self, data):
        all_items = set(np.arange(self.num_item))
        neg_items = {}
        for u in range(self.num_user):
            neg_items[u] = list(all_items - set(data.getrow(u).nonzero()[1]))

        return neg_items

ModuleNotFoundError: No module named 'utils.evaluation'